In [1]:
import os 
from logai.applications.openset.anomaly_detection.openset_anomaly_detection_workflow import OpenSetADWorkflowConfig, validate_config_dict
from logai.utils.file_utils import read_file
from logai.utils.dataset_utils import split_train_dev_test_for_anomaly_detection
import logging 
from logai.dataloader.data_loader import FileDataLoader
from logai.preprocess.bgl_preprocessor import BGLPreprocessor
from logai.information_extraction.log_parser import LogParser
from logai.preprocess.openset_partitioner import OpenSetPartitioner
from logai.analysis.nn_anomaly_detector import NNAnomalyDetector
from logai.information_extraction.log_vectorizer import LogVectorizer
from logai.utils import constants

In [2]:
config_path = "bgl_logbert_config.yaml"
config_parsed = read_file(config_path)
config_dict = config_parsed["workflow_config"]
validate_config_dict(config_dict)
config = OpenSetADWorkflowConfig.from_dict(config_dict)

In [3]:
dataloader = FileDataLoader(config.data_loader_config)
logrecord = dataloader.load_data()
print (logrecord.body[constants.LOGLINE_NAME])

0         RAS KERNEL INFO instruction cache parity error...
1         RAS KERNEL INFO instruction cache parity error...
2         RAS KERNEL INFO instruction cache parity error...
3         RAS KERNEL INFO instruction cache parity error...
4         RAS KERNEL INFO instruction cache parity error...
                                ...                        
358455    RAS KERNEL FATAL idoproxy communication failur...
358456    RAS KERNEL FATAL idoproxy communication failur...
358457    RAS KERNEL FATAL idoproxy communication failur...
358458    RAS KERNEL FATAL idoproxy communication failur...
358459    RAS KERNEL FATAL idoproxy communication failur...
Name: logline, Length: 358460, dtype: object


/Users/amrita.saha/Home/salesforce/workspace/code/AIOps/RCA_Log/logai_opensource/logai/logai/dataloader/data_loader.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected[constants.LOG_TIMESTAMPS] = pd.to_datetime(


In [4]:
preprocessor = BGLPreprocessor(config.preprocessor_config)
preprocessed_filepath = os.path.join(config.output_dir, 'BGL_11k_processed.csv')            
logrecord = preprocessor.clean_log(logrecord)
logrecord.save_to_csv(preprocessed_filepath)
print (logrecord.body[constants.LOGLINE_NAME])

0         RAS KERNEL INFO instruction cache parity error...
1         RAS KERNEL INFO instruction cache parity error...
2         RAS KERNEL INFO instruction cache parity error...
3         RAS KERNEL INFO instruction cache parity error...
4         RAS KERNEL INFO instruction cache parity error...
                                ...                        
358455    RAS KERNEL FATAL idoproxy communication failur...
358456    RAS KERNEL FATAL idoproxy communication failur...
358457    RAS KERNEL FATAL idoproxy communication failur...
358458    RAS KERNEL FATAL idoproxy communication failur...
358459    RAS KERNEL FATAL idoproxy communication failur...
Name: logline, Length: 358460, dtype: object


In [5]:
partitioner = OpenSetPartitioner(config.open_set_partitioner_config)
partitioned_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session.csv')
logrecord = partitioner.partition(logrecord)
logrecord.save_to_csv(partitioned_filepath)
print (logrecord.body[constants.LOGLINE_NAME])

0       RAS KERNEL INFO instruction cache parity error...
1       RAS KERNEL INFO instruction cache parity error...
2       RAS KERNEL INFO instruction cache parity error...
3       RAS KERNEL INFO instruction cache parity error...
4       RAS KERNEL INFO instruction cache parity error...
                              ...                        
1848    RAS APP FATAL ciod Error reading message prefi...
1849    RAS KERNEL FATAL Lustre mount FAILED ALPHANUM ...
1850    RAS KERNEL FATAL Lustre mount FAILED ALPHANUM ...
1851    RAS KERNEL FATAL Lustre mount FAILED ALPHANUM ...
1852    RAS KERNEL FATAL idoproxy communication failur...
Name: logline, Length: 1853, dtype: object


In [6]:
train_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session_supervised_train.csv')
dev_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session_supervised_dev.csv')
test_filepath = os.path.join(config.output_dir, 'BGL_11k_nonparsed_session_supervised_test.csv')

(train_data, dev_data, test_data) = split_train_dev_test_for_anomaly_detection(
                logrecord,training_type=config.training_type,
                test_data_frac_neg_class=config.test_data_frac_neg,
                test_data_frac_pos_class=config.test_data_frac_pos,
                shuffle=config.train_test_shuffle
            )

train_data.save_to_csv(train_filepath)
dev_data.save_to_csv(dev_filepath)
test_data.save_to_csv(test_filepath)
print ('Train/Dev/Test Anomalous', len(train_data.labels[train_data.labels[constants.LABELS]==1]), 
                                   len(dev_data.labels[dev_data.labels[constants.LABELS]==1]), 
                                   len(test_data.labels[test_data.labels[constants.LABELS]==1]))
print ('Train/Dev/Test Normal', len(train_data.labels[train_data.labels[constants.LABELS]==0]), 
                                   len(dev_data.labels[dev_data.labels[constants.LABELS]==0]), 
                                   len(test_data.labels[test_data.labels[constants.LABELS]==0]))

indices_train/dev/test:  40 5 1808
Train/Dev/Test Anomalous 0 0 1808
Train/Dev/Test Normal 40 5 0


In [7]:
vectorizer = LogVectorizer(config.log_vectorizer_config)
vectorizer.fit(train_data)
train_features = vectorizer.transform(train_data)
dev_features = vectorizer.transform(dev_data)
test_features = vectorizer.transform(test_data)
print (train_features)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 40
})


In [9]:
anomaly_detector = NNAnomalyDetector(config=config.nn_anomaly_detection_config)
anomaly_detector.fit(train_features, dev_features)

loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file vocab.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


initialized data collator


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/Users/amrita.saha/opt/anaconda3/envs/loglib/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 40
  Num Epochs = 10
  Instantaneous batch size per device = 4

Step,Training Loss,Validation Loss
50,3.082700,3.418151
100,2.826200,3.554583


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 256
Saving model checkpoint to temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-50
Configuration saved in temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-50/config.json
Model weights saved in temp_output/BGL_11k_parsed_session_supervised_AD/bert-base-cased/checkpoint-50/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__. If __index_level_0__ are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5
  Batch size = 256
Saving mod

In [ ]:
predict_results = anomaly_detector.predict(test_features)
print (predict_results)